## **MACHINE LEARNING PRODUCTS (MLOps)**


***En este cuaderno jupiter se encuentra un producto minimo viable, de nuestros proyecto - "Sistema de Recomendacion del Mundial del 2026" solo vamos  a utilizar solo la data de Google Maps***

In [1]:
# Librerias
import pymysql
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
import os

In [2]:
# La data se encuentra en AWS vamos a conectar
# Cargamos el archivo .env
load_dotenv()

# Acceder a las variables entorno
password_bd = os.getenv("PASS_BD")
user_bd = os.getenv("USER_BD")
host_name = os.getenv("HOST_NAME")
name_bd = os.getenv("NAME_BD")

# Conectamos la base de datos


def BD_connection(host_name, user_bd, user_pass, name_bd):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_bd,
            passwd=user_pass,
            database=name_bd,
            port=3306
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [3]:
connection = BD_connection(host_name, user_bd, password_bd, name_bd)

Error: '2003 (HY000): Can't connect to MySQL server on 'mysql-db.cpmoyou2oi10.us-west-1.rds.amazonaws.com:3306' (10060)'
